Este código implementa o PRISM (PRobabilistic Incremental Subsequence Mining), que é baseado em regras e visa aprender conjuntos de regras a partir de um conjunto de dados. O PRISM constrói regras de classificação para cada classe, combinando-as posteriormente num conjunto de regras para realizar a classificação. Foram implementadas as seguintes funções:

calculate_probabilities(data, target, target_value): Esta função tem como objetivo calcular as probabilidades condicionais de um determinado exemplo pertencer à classe target_value, tendo em conta os valores de cada atributo presentes nos dados. O resultado é uma lista de tuplas que inclui a probabilidade, o índice da coluna e o valor do atributo em questão.

induce_rule(data, target, target_value): Através de um processo iterativo, esta função gera uma única regra para a classe target_value. Em cada iteração, é selecionada a combinação de atributo e valor que apresenta a maior probabilidade condicional de pertencer à classe target_value, e esta combinação é adicionada à regra atual. O processo é repetido até que todos os exemplos restantes pertençam à classe target_value. A regra final é retornada na forma de uma lista de tuplas, onde cada tupla inclui o índice da coluna e o valor do atributo.

prism(data, target, classes): Esta é a função principal do algoritmo PRISM, a qual recebe como entrada os dados, as classes correspondentes e uma lista de classes distintas. A função itera por cada classe, e enquanto existirem exemplos não classificados para essa classe, gera uma regra através da função induce_rule, atualizando posteriormente os dados e classes restantes, removendo os exemplos classificados pela regra gerada. As regras geradas para cada classe são armazenadas num dicionário e retornadas ao final.

In [5]:
import numpy as np

def calculate_probabilities(data, target, target_value):
    # Inicializa uma lista para armazenar as probabilidades
    probabilities = []
    
    # Loop por cada coluna na tabela de dados
    for col in range(data.shape[1]):
        # Encontra os valores distintos e as contagens para a coluna atual
        unique_values, counts = np.unique(data[:, col], return_counts=True)
        
        # Loop por cada valor distinto
        for value in unique_values:
            # Seleciona os dados correspondentes ao valor atual
            subset = data[data[:, col] == value]
            # Seleciona as classes correspondentes ao valor atual
            target_subset = target[data[:, col] == value]
            # Conta o número de exemplos na classe `target_value` no subconjunto
            class_count = np.sum(target_subset == target_value)
            # Calcula a probabilidade de um exemplo estar na classe `target_value` dado o valor `value` para o atributo `col`
            prob = class_count / len(target_subset)
            # Adiciona a probabilidade, o índice da coluna e o valor do atributo à lista de probabilidades
            probabilities.append((prob, col, value))
    # Retorna a lista de probabilidades
    return probabilities

def induce_rule(data, target, target_value):
    # Inicializa uma lista vazia para armazenar a regra
    rule = []
    # Faz uma cópia dos dados e das classes para evitar modificações nas variáveis originais
    remaining_data = data.copy()
    remaining_target = target.copy()

    # Loop enquanto houver exemplos não classificados na classe `target_value`
    while True:
        # Calcula as probabilidades para a classe `target_value` com base nos dados e nas classes restantes
        probabilities = calculate_probabilities(remaining_data, remaining_target, target_value)
        # Encontra a probabilidade máxima, o índice da coluna e o valor do atributo
        max_prob, max_col, max_value = max(probabilities, key=lambda x: x[0])

        # Cria uma máscara para selecionar os exemplos correspondentes ao valor máximo
        mask = remaining_data[:, max_col] == max_value
        # Atualiza os dados e as classes restantes com base na máscara
        remaining_data = remaining_data[mask]
        remaining_target = remaining_target[mask]

        # Adiciona o critério de decisão à regra
        rule.append((max_col, max_value))

        # Se todos os exemplos restantes estão na classe `target_value`, sai do loop
        if np.all(remaining_target == target_value):
            break
    # Retorna a regra gerada
    return rule

def prism(data, target, classes):
    # Inicializa um dicionário para armazenar as regras para cada classe
    rules = {i: [] for i in classes}
    # Faz uma cópia dos dados e das classes para evitar modificações nas variáveis originais
    data_remaining = data.copy()
    target_remaining = target.copy()
    # Loop por cada classe
    for class_value in classes:
        # Loop enquanto houver exemplos não classificados na classe `class_value`
        while True:
            # Gera uma regra para a classe `class_value` com base nos dados e nas classes restantes
            rule = induce_rule(data_remaining, target_remaining, class_value)
            # Adiciona a regra gerada à lista de regras para a classe `class_value`
            rules[class_value].append(rule)

            # Cria uma máscara para selecionar os exemplos que não satisfazem a regra gerada
            indices_to_remove = np.array([True] * len(target_remaining))
            for col, value in rule:
                indices_to_remove = np.logical_and(indices_to_remove, data_remaining[:, col] == value)

            # Atualiza os dados e as classes restantes com base na máscara
            data_remaining = data_remaining[~indices_to_remove]
            target_remaining = target_remaining[~indices_to_remove]

            # Se não há mais exemplos na classe `class_value`, sai do loop
            if np.sum(target_remaining == class_value) == 0:
                break

    # Retorna o dicionário de regras
    return rules

**Exemplo de uso**

In [9]:
#Cria um conjunto de dados de exemplo e atribui as classes correspondentes
data = np.array([[1, 1, 0],
                 [1, 0, 1],
                 [0, 1, 1],
                 [0, 0, 0]])

target = np.array([1, 1, 0, 0])

#Identifiqua todas as classes distintas no conjunto de dados
classes = np.unique(target)

#Aplica a função PRISM para gerar as regras
rules = prism(data, target, classes)

for class_value, rule_set in rules.items():
    print(f"Regras para a classe {class_value}:")
    for rule in rule_set:
        print(f"  SE", end=" ")
        for i, (col, value) in enumerate(rule):
            print(f"Atributo {col} == {value}", end="")
            if i < len(rule) - 1:
                print(" E", end=" ")
        print(f" ENTÃO Classe = {class_value}")
    print()

Regras para a classe 0:
  SE Atributo 0 == 0 ENTÃO Classe = 0

Regras para a classe 1:
  SE Atributo 0 == 1 ENTÃO Classe = 1

